In [1]:
import json
import os
import time
import requests

In [2]:
import psycopg2
import os
import json

database_dict = {
        "database": os.environ.get("POSTGRES_DB"),
        "user": os.environ.get("POSTGRES_USERNAME"),
        "password": os.environ.get("POSTGRES_PASSWORD"),
        "host": os.environ.get("POSTGRES_WRITER"),
        "port": os.environ.get("POSTGRES_PORT"),
    }
engine = psycopg2.connect(**database_dict)
cur = engine.cursor()

def get_xDai_daos(cur, engine):
    execute_string = f"SELECT * FROM daohaus.dao"
    cur.execute(execute_string)
    records = cur.fetchall()
    js = [];
    for x in records:
        if(x[5] == 392):
            curl = []
            curl.append(x[0])
            curl.append(x[1])
            js.append(curl)
    return js

xDai_net_daos = get_xDai_daos(cur, engine)

In [5]:
md_of_all_dao_tokens_and_balances_xDai = {}
all_tokens = set()

In [ ]:
for datapoint in xDai_net_daos:
    dao = str(datapoint[1])
    response = requests.get(f"https://blockscout.com/xdai/mainnet/api?module=account&action=tokenlist&address={dao}")
    test_data = response.json()
    if(test_data["message"] == "OK"):
        list_tokens_and_data = []
        for token in test_data["result"]:
            cd = {}
            cd['contractAddress'] = token['contractAddress']
            cd['balance'] = token['balance']
            cd['divisor'] = token['decimals']
            cd['name'] = token['name']
            list_tokens_and_data.append(cd)
            all_tokens.add(token['contractAddress'])
        
        md_of_all_dao_tokens_and_balances_xDai[str(dao)] = list_tokens_and_data
    elif (test_data["message"] == "No tokens found"):
        md_of_all_dao_tokens_and_balances_xDai[str(dao)] = "NO TOKENS"
    time.sleep(1)
    print(dao, test_data["message"])

In [10]:
js_str = json.dumps(md_of_all_dao_tokens_and_balances_xDai)
js_fin = json.loads(js_str)
with open('./all_xDai_dao_treasury_with_token_balances.json', 'w') as f:
        json.dump(js_fin, f)